In [1]:
ls

ATLAS_R2.0/            OpenDataset/         datapreprocessing_ATLAS.ipynb*
Dataset_Brain_lesion/  Processed_data_nii/  datapreprocessing_MnMCardiac.ipynb*
Dataset_Prostate/      datafile/            datapreprocessing_Prostate.ipynb*


In [2]:
# data path
datadownloadpath_Train = './OpenDataset/Training/'
datadownloadpath_Val = './OpenDataset/Validation/'
datadownloadpath_Test = './OpenDataset/Testing/'
datasavepath = './OpenDataset/MnMCardiac/'
metadata = './OpenDataset/211230_M&Ms_Dataset_information_diagnosis_opendataset.csv'

In [3]:
import glob
import SimpleITK as sitk
import numpy as np
import pandas as pd
import random
import nibabel as nib
from collections import Counter
import os

In [6]:
metaload = pd.read_csv(metadata)
groups = Counter(metaload['Centre'])

In [7]:
groups

Counter({1: 95, 4: 75, 2: 74, 3: 51, 5: 50})

In [8]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    resample.SetUseNearestNeighborExtrapolator(True)
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj

In [9]:
# just use this split.
imgs_train = glob.glob(datadownloadpath_Train + "*/*/*_sa.nii.gz")
imgs_train = [ fid for fid in sorted(imgs_train) ]
imgs_val = glob.glob(datadownloadpath_Val + "*/*_sa.nii.gz")
imgs_val = [ fid for fid in sorted(imgs_val) ]
imgs_test = glob.glob(datadownloadpath_Test + "*/*_sa.nii.gz")
imgs_test = [ fid for fid in sorted(imgs_test) ]
imgs = imgs_train + imgs_val + imgs_test

segs_train = glob.glob(datadownloadpath_Train + "*/*/*_sa_gt.nii.gz")
segs_train = [ fid for fid in sorted(segs_train) ]
segs_val = glob.glob(datadownloadpath_Val + "*/*_sa_gt.nii.gz")
segs_val = [ fid for fid in sorted(segs_val) ]
segs_test = glob.glob(datadownloadpath_Test + "*/*_sa_gt.nii.gz")
segs_test = [ fid for fid in sorted(segs_test) ]
segs = segs_train + segs_val + segs_test


pids = [pid.split("/")[-1].split("_")[0] for pid in imgs]

In [10]:
imgs

['./OpenDataset/Training/Labeled/A0S9V9/A0S9V9_sa.nii.gz',
 './OpenDataset/Training/Labeled/A1D0Q7/A1D0Q7_sa.nii.gz',
 './OpenDataset/Training/Labeled/A1D9Z7/A1D9Z7_sa.nii.gz',
 './OpenDataset/Training/Labeled/A1E9Q1/A1E9Q1_sa.nii.gz',
 './OpenDataset/Training/Labeled/A1O8Z3/A1O8Z3_sa.nii.gz',
 './OpenDataset/Training/Labeled/A2C0I1/A2C0I1_sa.nii.gz',
 './OpenDataset/Training/Labeled/A2N8V0/A2N8V0_sa.nii.gz',
 './OpenDataset/Training/Labeled/A3B7E5/A3B7E5_sa.nii.gz',
 './OpenDataset/Training/Labeled/A3H1O5/A3H1O5_sa.nii.gz',
 './OpenDataset/Training/Labeled/A4B5U4/A4B5U4_sa.nii.gz',
 './OpenDataset/Training/Labeled/A4J4S4/A4J4S4_sa.nii.gz',
 './OpenDataset/Training/Labeled/A4U9V5/A4U9V5_sa.nii.gz',
 './OpenDataset/Training/Labeled/A5E0T8/A5E0T8_sa.nii.gz',
 './OpenDataset/Training/Labeled/A6B5G9/A6B5G9_sa.nii.gz',
 './OpenDataset/Training/Labeled/A6D5F9/A6D5F9_sa.nii.gz',
 './OpenDataset/Training/Labeled/A6M1Q7/A6M1Q7_sa.nii.gz',
 './OpenDataset/Training/Labeled/A7D9L8/A7D9L8_sa.nii.gz

In [11]:
segs

['./OpenDataset/Training/Labeled/A0S9V9/A0S9V9_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A1D0Q7/A1D0Q7_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A1D9Z7/A1D9Z7_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A1E9Q1/A1E9Q1_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A1O8Z3/A1O8Z3_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A2C0I1/A2C0I1_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A2N8V0/A2N8V0_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A3B7E5/A3B7E5_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A3H1O5/A3H1O5_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A4B5U4/A4B5U4_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A4J4S4/A4J4S4_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A4U9V5/A4U9V5_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A5E0T8/A5E0T8_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A6B5G9/A6B5G9_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A6D5F9/A6D5F9_sa_gt.nii.gz',
 './OpenDataset/Training/Labeled/A6M1Q7/A6M1Q7_sa_gt.nii.gz',
 './Open

In [12]:
pids

['A0S9V9',
 'A1D0Q7',
 'A1D9Z7',
 'A1E9Q1',
 'A1O8Z3',
 'A2C0I1',
 'A2N8V0',
 'A3B7E5',
 'A3H1O5',
 'A4B5U4',
 'A4J4S4',
 'A4U9V5',
 'A5E0T8',
 'A6B5G9',
 'A6D5F9',
 'A6M1Q7',
 'A7D9L8',
 'A7G0P5',
 'A7M7P8',
 'A7O4T6',
 'A8C9U8',
 'A8E1F4',
 'A9C5P4',
 'A9E3G9',
 'A9J5Q7',
 'A9J8W7',
 'B0I2Z0',
 'B0N3W8',
 'B2C2Z7',
 'B2D9M2',
 'B2D9O2',
 'B2F4K5',
 'B2G5R2',
 'B3D0N1',
 'B3O1S0',
 'B3P3R1',
 'B4O3V3',
 'B6D0U7',
 'B8H5H6',
 'B8J7R4',
 'B9E0Q1',
 'B9O1Q0',
 'C0K1P0',
 'C0S7W0',
 'C1G5Q0',
 'C1K8P5',
 'C2J0K3',
 'C2L5P7',
 'C2M6P8',
 'C3I2K3',
 'C4R8T7',
 'C4S8W9',
 'C5M4S2',
 'C6J5P1',
 'C8P3S7',
 'D0H9I4',
 'D0R0R9',
 'D1J5P6',
 'D1L4Q9',
 'D1M1S6',
 'D3D4Y5',
 'D3F3O5',
 'D3F9H9',
 'D3O9U9',
 'D4M3Q2',
 'D4N6W6',
 'D6H6O2',
 'D8E4F4',
 'D9L1Z3',
 'E0M3U7',
 'E0O0S0',
 'E3T0Z2',
 'E4M2Q7',
 'E4W8Z7',
 'E5E6O8',
 'E5F5V7',
 'E9H1U4',
 'E9H2K7',
 'E9L1W5',
 'E9V4Z8',
 'F0J2R8',
 'F1F3I6',
 'F2H5S1',
 'F3G5K5',
 'F4K3S1',
 'F5I9Q2',
 'F8N2S1',
 'G0H4J3',
 'G0I6P3',
 'G1N6S7',
 'G2J1M5',

## Step 0, Generate slices

In [13]:
savefold = './Dataset_Cardiac/'
if os.path.exists(savefold) == False:
    os.mkdir(savefold)

In [14]:
metaload = pd.read_csv(metadata)

In [15]:
for (imgfile, lblfile, pid) in zip(imgs, segs, pids):
    print('Processing Case ' + pid)
    lblload = nib.load(lblfile)
    lbl = lblload.get_fdata()
    
    # get the metainfo for saving path
    metaload_group = metaload[metaload['External code'] == pid]['Centre'].tolist()[0]
    
    saving_group = savefold + str(metaload_group)
    if os.path.exists(saving_group) == False:
        os.mkdir(saving_group)
    
    for n_frame in range(lbl.shape[-1]):
        
        if np.sum(lbl[:, :, :, n_frame]) > 0: # this time frame is labeled,
            
            saving_pid_group = saving_group + '/' + pid + '_' + str(n_frame)
            if os.path.exists(saving_pid_group) == False:
                os.mkdir(saving_pid_group)
        
            Imgload = nib.load(imgfile)
            Imgc1 = Imgload.get_fdata() 

            affine_origin = Imgload.affine
#             affine_origin = np.eye(4)
    
            newLabelImg = nib.Nifti1Image(Imgc1[:, :, :, n_frame], affine_origin)
            newLabelImg.set_qform(newLabelImg.get_qform())
            newLabelImg.set_sform(newLabelImg.get_sform())
            imgsavepath = saving_pid_group + '/image.nii.gz'
            nib.save(newLabelImg, imgsavepath)

            newLabelLbl = nib.Nifti1Image(lbl[:, :, :, n_frame], affine_origin)
            newLabelLbl.set_qform(newLabelLbl.get_qform())
            newLabelLbl.set_sform(newLabelLbl.get_sform())
            lblsavepath = saving_pid_group + '/seg.nii.gz'
            nib.save(newLabelLbl, lblsavepath)
    
    print('Data shape ' + str(np.array(lbl[:, :, :, n_frame].shape)))
    print('Case ' + pid + ' saved')

Processing Case A0S9V9
Data shape [216 256  13]
Case A0S9V9 saved
Processing Case A1D0Q7
Data shape [320 320  10]
Case A1D0Q7 saved
Processing Case A1D9Z7
Data shape [208 256  13]
Case A1D9Z7 saved
Processing Case A1E9Q1
Data shape [208 256  13]
Case A1E9Q1 saved
Processing Case A1O8Z3
Data shape [256 256  11]
Case A1O8Z3 saved
Processing Case A2C0I1
Data shape [208 256  12]
Case A2C0I1 saved
Processing Case A2N8V0
Data shape [196 240  11]
Case A2N8V0 saved
Processing Case A3B7E5
Data shape [320 320  10]
Case A3B7E5 saved
Processing Case A3H1O5
Data shape [196 240  11]
Case A3H1O5 saved
Processing Case A4B5U4
Data shape [196 240  14]
Case A4B5U4 saved
Processing Case A4J4S4
Data shape [208 256  11]
Case A4J4S4 saved
Processing Case A4U9V5
Data shape [196 240  12]
Case A4U9V5 saved
Processing Case A5E0T8
Data shape [256 256  11]
Case A5E0T8 saved
Processing Case A6B5G9
Data shape [196 240  12]
Case A6B5G9 saved
Processing Case A6D5F9
Data shape [216 256  13]
Case A6D5F9 saved
Processing

Data shape [256 256  12]
Case C7M6W0 saved
Processing Case C8O0P2
Data shape [256 256  10]
Case C8O0P2 saved
Processing Case D1H2O9
Data shape [256 256  13]
Case D1H2O9 saved
Processing Case D1L6T4
Data shape [512 440  12]
Case D1L6T4 saved
Processing Case D3K5Q2
Data shape [256 256   8]
Case D3K5Q2 saved
Processing Case D3Q0W9
Data shape [288 288  10]
Case D3Q0W9 saved
Processing Case D6E9U8
Data shape [320 320  10]
Case D6E9U8 saved
Processing Case D9F5P1
Data shape [208 256   9]
Case D9F5P1 saved
Processing Case D9I8O7
Data shape [288 288  12]
Case D9I8O7 saved
Processing Case E0J2Z9
Data shape [512 440  14]
Case E0J2Z9 saved
Processing Case E0J7L9
Data shape [208 256   9]
Case E0J7L9 saved
Processing Case E1L7M3
Data shape [480 436  13]
Case E1L7M3 saved
Processing Case E3F5U2
Data shape [256 256  11]
Case E3F5U2 saved
Processing Case E3L8U8
Data shape [288 288  12]
Case E3L8U8 saved
Processing Case E4H7L4
Data shape [512 440  15]
Case E4H7L4 saved
Processing Case E4I9O7
Data shape

## Step 1, Data resample

In [16]:
# get the volume list
imgs_volume = glob.glob(savefold + "*/*/image.nii.gz")
imgs_volume = [ fid for fid in sorted(imgs_volume) ]

segs_volume = glob.glob(savefold + "*/*/seg.nii.gz")
segs_volume = [ fid for fid in sorted(segs_volume) ]

pids_volume = [pids_volume.split("/")[-2] for pids_volume in imgs_volume]

In [17]:
len(imgs_volume)

690

In [18]:
len(segs_volume)

690

In [19]:
len(pids_volume)

690

In [20]:
targetspacing = [1.25, 1.25, 10.]

In [21]:
for (imgfile, lblfile, pid) in zip(imgs_volume, segs_volume, pids_volume):
    print(imgfile)
    print(lblfile)
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( lblfile )
    img_spa_ori = img_obj.GetSpacing()
    seg_obj.SetSpacing(img_spa_ori)
    res_img_o = resample_by_res(img_obj, targetspacing, interpolator = sitk.sitkLinear,
                                    logging = True)
    res_lb_o = resample_lb_by_res(seg_obj, targetspacing, interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)

    sitk.WriteImage(res_img_o, imgfile, True) 
    sitk.WriteImage(res_lb_o, lblfile, True) 

./Dataset_Cardiac/1/A0S9V9_0/image.nii.gz
./Dataset_Cardiac/1/A0S9V9_0/seg.nii.gz
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Label values: [0. 1. 2. 3.]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
./Dataset_Cardiac/1/A0S9V9_9/image.nii.gz
./Dataset_Cardiac/1/A0S9V9_9/seg.nii.gz
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936

./Dataset_Cardiac/1/A6D5F9_0/image.nii.gz
./Dataset_Cardiac/1/A6D5F9_0/seg.nii.gz
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
./Dataset_Cardiac/1/A6D5F9_11/image.nii.gz
./Dataset_Cardiac/1/A6D5F9_11/seg.nii.gz
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [216.  256.   10.4]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25,

./Dataset_Cardiac/1/B2C2Z7_0/image.nii.gz
./Dataset_Cardiac/1/B2C2Z7_0/seg.nii.gz
Spacing: (1.25, 1.25, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 16) -> [196.         240.          15.36000061]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 16) -> [196.         240.          15.36000061]
Spacing: (1.25, 1.25, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 16) -> [196.         240.          15.36000061]
Spacing: (1.25, 1.25, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 16) -> [196.         240.          15.36000061]
Spacing: (1.25, 1.25, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 16) -> [196.         240.          15.36000061]
./Dataset_Cardiac/1/B2C2Z7_8/image.nii.gz
./Dataset_Cardiac/1/B2C2Z7_8/seg.nii.gz
Spacing: (1.25, 1.25, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 16) -> [196.         240.          15.36000061]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.

./Dataset_Cardiac/1/B4O3V3_0/image.nii.gz
./Dataset_Cardiac/1/B4O3V3_0/seg.nii.gz
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
./Dataset_Cardiac/1/B4O3V3_6/image.nii.gz
./Dataset_Cardiac/1/B4O3V3_6/seg.nii.gz
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10) -> [208. 256.   8.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 10)

./Dataset_Cardiac/1/B9H8N8_10/image.nii.gz
./Dataset_Cardiac/1/B9H8N8_10/seg.nii.gz
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 12) -> [222.1333271  271.99999237  11.52000046]
Label values: [0. 1. 2. 3.]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 12) -> [222.1333271  271.99999237  11.52000046]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 12) -> [222.1333271  271.99999237  11.52000046]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 12) -> [222.1333271  271.99999237  11.52000046]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 12) -> [222.1333271  271.99999237  11.52000046]
./Dataset_Cardiac/1/C0K1P0_0/image.nii.gz
./Dataset_Cardiac/1/C0K1P0_0/seg.nii.gz
Spacing: (1.4166666269302368, 1

...

./Dataset_Cardiac/5/G8K0M3_10/image.nii.gz
./Dataset_Cardiac/5/G8K0M3_10/seg.nii.gz
Spacing: (0.7955000400543213, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83681641 280.01599312  13.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7955000400543213, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83681641 280.01599312  13.        ]
Spacing: (0.7955000400543213, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83681641 280.01599312  13.        ]
Spacing: (0.7955000400543213, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83681641 280.01599312  13.        ]
Spacing: (0.7955000400543213, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83681641 280.01599312  13.        ]
./Dataset_Cardiac/5/G8K0M3_25/image.nii.gz
./Dataset_Cardiac/5/G8K0M3_25/seg.nii.gz
Spacing: (0.7955000400543213, 0.7954999804496765, 10.0) -> [1.

./Dataset_Cardiac/5/J6M5O2_11/image.nii.gz
./Dataset_Cardiac/5/J6M5O2_11/seg.nii.gz
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.0160141   11.        ]
Label values: [0. 1. 2. 3.]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.0160141   11.        ]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.0160141   11.        ]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.0160141   11.        ]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.0160141   11.        ]
./Dataset_Cardiac/5/J6M5O2_33/image.nii.gz
./Dataset_Cardiac/5/J6M5O2_33/seg.nii.gz
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 

./Dataset_Cardiac/5/L5U7Y4_10/image.nii.gz
./Dataset_Cardiac/5/L5U7Y4_10/seg.nii.gz
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83679199 280.0160141   13.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83679199 280.0160141   13.        ]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83679199 280.0160141   13.        ]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83679199 280.0160141   13.        ]
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 13) -> [325.83679199 280.0160141   13.        ]
./Dataset_Cardiac/5/L5U7Y4_33/image.nii.gz
./Dataset_Cardiac/5/L5U7Y4_33/seg.nii.gz
Spacing: (0.7954999804496765, 0.7955000400543213, 10.0) -> [1.

./Dataset_Cardiac/5/N7P3T8_10/image.nii.gz
./Dataset_Cardiac/5/N7P3T8_10/seg.nii.gz
Spacing: (0.7955000400543213, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83681641 280.0160141   14.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7955000400543213, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83681641 280.0160141   14.        ]
Spacing: (0.7955000400543213, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83681641 280.0160141   14.        ]
Spacing: (0.7955000400543213, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83681641 280.0160141   14.        ]
Spacing: (0.7955000400543213, 0.7955000400543213, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83681641 280.0160141   14.        ]
./Dataset_Cardiac/5/N7P3T8_25/image.nii.gz
./Dataset_Cardiac/5/N7P3T8_25/seg.nii.gz
Spacing: (0.7955000400543213, 0.7955000400543213, 10.0) -> [1.

./Dataset_Cardiac/5/P8V0Y7_27/image.nii.gz
./Dataset_Cardiac/5/P8V0Y7_27/seg.nii.gz
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Label values: [0. 1. 2. 3.]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
./Dataset_Cardiac/5/P8V0Y7_9/image.nii.gz
./Dataset_Cardiac/5/P8V0Y7_9/seg.nii.gz
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14

## Step 2, Intensity Normalization

In [22]:
for (imgfile, pid) in zip(imgs_volume, pids_volume):
    
    img_obj = sitk.ReadImage( imgfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    pixel_mean = np.mean(array)
    pixel_std = np.std(array)
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    sitk.WriteImage(normalized_img, imgfile, True) 
    print('Id ' + pid + ' normalized')

Id A0S9V9_0 normalized
Id A0S9V9_9 normalized
Id A1D9Z7_11 normalized
Id A1D9Z7_22 normalized
Id A1E9Q1_0 normalized
Id A1E9Q1_9 normalized
Id A2C0I1_0 normalized
Id A2C0I1_7 normalized
Id A2L1N6_0 normalized
Id A2L1N6_12 normalized
Id A2N8V0_0 normalized
Id A2N8V0_9 normalized
Id A3H1O5_0 normalized
Id A3H1O5_12 normalized
Id A4B5U4_0 normalized
Id A4B5U4_10 normalized
Id A4J4S4_0 normalized
Id A4J4S4_7 normalized
Id A4U9V5_0 normalized
Id A4U9V5_8 normalized
Id A6B5G9_0 normalized
Id A6B5G9_11 normalized
Id A6D5F9_0 normalized
Id A6D5F9_11 normalized
Id A7D9L8_0 normalized
Id A7D9L8_11 normalized
Id A7M7P8_0 normalized
Id A7M7P8_9 normalized
Id A7O4T6_0 normalized
Id A7O4T6_10 normalized
Id B0H7V0_0 normalized
Id B0H7V0_9 normalized
Id B0I2Z0_0 normalized
Id B0I2Z0_8 normalized
Id B2C2Z7_0 normalized
Id B2C2Z7_8 normalized
Id B2D9M2_0 normalized
Id B2D9M2_8 normalized
Id B2G5R2_0 normalized
Id B2G5R2_7 normalized
Id B3F0V9_0 normalized
Id B3F0V9_12 normalized
Id B3O1S0_0 normalized
I

## Generate datafiles.

In [23]:
datafilepath = './datafile/Dataset_Cardiac/'
training_folder = '1'
if os.path.exists(datafilepath) == False:
    os.mkdir(datafilepath)

In [24]:
domainlists = glob.glob(savefold + '*/')

In [25]:
domainlists

['./Dataset_Cardiac/3/',
 './Dataset_Cardiac/5/',
 './Dataset_Cardiac/2/',
 './Dataset_Cardiac/4/',
 './Dataset_Cardiac/1/']

In [26]:
# just utilize the generated datafiles.